In [1]:
import torch
import torchvision
from torch.utils.data import DataLoader, Dataset
from torch import nn
from torch.nn import functional as F
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

DEVICE = torch.device('cuda') if torch.cuda.is_available() else 'cpu'
print(f"Device: {DEVICE}")

train_dir = "../dev-data/train.csv"
X_train = pd.read_csv(train_dir).drop('label', axis = 1)
y_train = pd.read_csv(train_dir)['label']

Device: cuda
